In [33]:
import os

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from kornia.losses import focal_loss
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, WeightedRandomSampler

from models.hybrid_CNN_BiLSTM import HybridCNNBiLSTM
from models.simple_biLSTM import BiLSTMClassifier
from src.dataset_factory import DatasetFactory
from src.nn_trainer import NNTrainer
from src.nn_utils import JournalDataset, build_vocab

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Trabajando con: {device}")


factory = DatasetFactory()

Trabajando con: cpu


In [34]:
import pandas as pd

def perform_downsampling(df, target_column='label_idx'):
    
    counts = df[target_column].value_counts()
    min_class_size = counts.min() 
    
    target_size = max(min_class_size, 300) 
    
    downsampled_parts = []
    
    for class_index in counts.index:
        class_subset = df[df[target_column] == class_index]
        
        if len(class_subset) > target_size:
            
            downsampled_parts.append(class_subset.sample(target_size, random_state=42))
        else:
            
            downsampled_parts.append(class_subset)
            
    return pd.concat(downsampled_parts).sample(frac=1, random_state=42).reset_index(drop=True)


def perform_soft_downsampling(df, target_column='label_idx', ratio=2.0, min_floor=300):
    counts = df[target_column].value_counts()
    min_class_size = counts.min()
    
    
    base_size = max(min_class_size, min_floor)
    
    
    
    max_allowed_size = int(base_size * ratio)
    
    downsampled_parts = []
    
    for class_index in counts.index:
        class_subset = df[df[target_column] == class_index]
        n_samples = len(class_subset)
        
        if n_samples > max_allowed_size:
            
            downsampled_parts.append(class_subset.sample(max_allowed_size, random_state=42))
        else:
            
            downsampled_parts.append(class_subset)
            
    return pd.concat(downsampled_parts).sample(frac=1, random_state=42).reset_index(drop=True)

In [35]:
df = factory.create_dl_dataset('DL_Experiment_Base_aggresive')


TARGET_TEST_SIZE = 0.15
TARGET_VAL_SIZE = 0.15


train_val_df, test_df = train_test_split(
    df, test_size=TARGET_TEST_SIZE, stratify=df['label_idx'], random_state=42
)

val_relative_size = TARGET_VAL_SIZE / (1 - TARGET_TEST_SIZE)
train_df, val_df = train_test_split(
    train_val_df, test_size=val_relative_size, stratify=train_val_df['label_idx'], random_state=42
)

#perform downsampling
#train_df = perform_downsampling(train_df, target_column='label_idx')

print("Distribution after Downsampling:\n", train_df['label_idx'].value_counts())


vocab = build_vocab(train_df['processed_text'], max_features= 3000)

print(f"Total vocabulary size: {len(vocab)}")

print("Vocab sample:", list(vocab.items())[:10])


train_loader = DataLoader(
    JournalDataset(train_df['processed_text'], train_df['label_idx'], vocab), 
    batch_size=32, shuffle=True
)

val_loader = DataLoader(JournalDataset(val_df['processed_text'], val_df['label_idx'], vocab), batch_size=32, shuffle=False)
test_loader = DataLoader(JournalDataset(test_df['processed_text'], test_df['label_idx'], vocab), batch_size=32, shuffle=False)



Generating DL dataset for experiment: DL_Experiment_Base_aggresive...
Dataset DL and mapping saved in data/experiments/DL_Experiment_Base_aggresive
Distribution after Downsampling:
 label_idx
1    7169
2    3162
0     787
3     590
Name: count, dtype: int64
Total vocabulary size: 3002
Vocab sample: [('proposed', 2), ('model', 3), ('data', 4), ('method', 5), ('results', 6), ('learning', 7), ('performance', 8), ('methods', 9), ('based', 10), ('information', 11)]


In [36]:
#criterion = lambda preds, targets: focal_loss(preds, targets, alpha=0.5, gamma=2.0, reduction='mean')
criterion = nn.CrossEntropyLoss()

In [30]:

exp_path_simple = os.path.join('data', 'experiments', 'BiLSTM_Simple_V11')
os.makedirs(exp_path_simple, exist_ok=True)

model_simple = BiLSTMClassifier(
    vocab_size=len(vocab), 
    embed_dim=128, 
    hidden_dim=32, 
    output_dim=df['label_idx'].nunique()
).to(device)

optimizer = optim.Adam(model_simple.parameters(), lr=0.001, weight_decay=1e-5)

mode='max' 
factor=0.1 
patience=2 
scheduler = ReduceLROnPlateau(optimizer, mode=mode, factor=factor, patience=patience)


trainer_simple = NNTrainer(
    model=model_simple, 
    criterion=criterion, 
    optimizer=optimizer, 
    device=device, 
    exp_path=exp_path_simple,
    scheduler=scheduler 
)
trainer_simple.fit(train_loader, val_loader, epochs=30)


y_true, y_pred, acc = trainer_simple.evaluate(test_loader)
print(f"Test Accuracy: {acc:.3f}")

trainer_simple.save_results(y_true, y_pred, vocab)

trainer_simple.plot_learning_curves()

Starting training on cpu...
Epoch 01/30 | Loss: 0.065 | Val Acc: 0.617 -> Best model saved
Epoch 02/30 | Loss: 0.052 | Val Acc: 0.654 -> Best model saved
Epoch 03/30 | Loss: 0.041 | Val Acc: 0.693 -> Best model saved
Epoch 04/30 | Loss: 0.035 | Val Acc: 0.708 -> Best model saved
Epoch 05/30 | Loss: 0.030 | Val Acc: 0.757 -> Best model saved
Epoch 06/30 | Loss: 0.026 | Val Acc: 0.759 -> Best model saved
Epoch 07/30 | Loss: 0.025 | Val Acc: 0.796 -> Best model saved
Epoch 08/30 | Loss: 0.022 | Val Acc: 0.796
Epoch 09/30 | Loss: 0.020 | Val Acc: 0.803 -> Best model saved
Epoch 10/30 | Loss: 0.018 | Val Acc: 0.809 -> Best model saved
Epoch 11/30 | Loss: 0.017 | Val Acc: 0.812 -> Best model saved
Epoch 12/30 | Loss: 0.016 | Val Acc: 0.806
Epoch 13/30 | Loss: 0.015 | Val Acc: 0.807
Epoch 14/30 | Loss: 0.014 | Val Acc: 0.815 -> Best model saved
Epoch 15/30 | Loss: 0.014 | Val Acc: 0.811
Epoch 16/30 | Loss: 0.013 | Val Acc: 0.819 -> Best model saved
Epoch 17/30 | Loss: 0.012 | Val Acc: 0.817
E

In [37]:

exp_path_hybrid = os.path.join('data', 'experiments', 'Hybrid_CNN_BiLSTM_V11')
os.makedirs(exp_path_hybrid, exist_ok=True)

model_hybrid = HybridCNNBiLSTM(
    vocab_size=len(vocab), 
    embed_dim=128,      
    n_filters=100,       
    filter_sizes=[3, 4, 5], 
    hidden_dim=32,      
    output_dim=df['label_idx'].nunique(),
    dropout=0.6         
).to(device)


optimizer_h = optim.Adam(
    model_hybrid.parameters(), 
    lr=0.001, 
    weight_decay=1e-5  
)


scheduler = ReduceLROnPlateau(optimizer_h, mode=mode, factor=factor, patience=patience)
trainer_hybrid = NNTrainer(
    model=model_hybrid, 
    criterion=criterion, 
    optimizer=optimizer_h, 
    device=device, 
    exp_path=exp_path_hybrid,
    scheduler=scheduler
)


trainer_hybrid.fit(train_loader, val_loader, epochs=50)

Starting training on cpu...


c:\Python312\Lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.6 and num_layers=1
  warnings.warn(


Epoch 01/50 | Loss: 0.963 | Val Acc: 0.618 -> Best model saved
Epoch 02/50 | Loss: 0.791 | Val Acc: 0.682 -> Best model saved
Epoch 03/50 | Loss: 0.677 | Val Acc: 0.750 -> Best model saved
Epoch 04/50 | Loss: 0.583 | Val Acc: 0.785 -> Best model saved
Epoch 05/50 | Loss: 0.537 | Val Acc: 0.784
Epoch 06/50 | Loss: 0.485 | Val Acc: 0.807 -> Best model saved
Epoch 07/50 | Loss: 0.449 | Val Acc: 0.807
Epoch 08/50 | Loss: 0.414 | Val Acc: 0.802
Epoch 09/50 | Loss: 0.387 | Val Acc: 0.808 -> Best model saved
Epoch 10/50 | Loss: 0.354 | Val Acc: 0.796
Epoch 11/50 | Loss: 0.340 | Val Acc: 0.810 -> Best model saved
Epoch 12/50 | Loss: 0.308 | Val Acc: 0.795
Epoch 13/50 | Loss: 0.303 | Val Acc: 0.801
Epoch 14/50 | Loss: 0.278 | Val Acc: 0.812 -> Best model saved
Epoch 15/50 | Loss: 0.250 | Val Acc: 0.808
Epoch 16/50 | Loss: 0.231 | Val Acc: 0.804
Epoch 17/50 | Loss: 0.223 | Val Acc: 0.809
Epoch 18/50 | Loss: 0.180 | Val Acc: 0.808
Epoch 19/50 | Loss: 0.173 | Val Acc: 0.810

[Early Stopping] The m

In [38]:

y_true_h, y_pred_h, acc_h = trainer_hybrid.evaluate(test_loader)
trainer_hybrid.save_results(y_true_h, y_pred_h, vocab)
print(f"Test Accuracy Hybrid: {acc_h:.3f}")
trainer_hybrid.plot_learning_curves()

Results saved in data\experiments\Hybrid_CNN_BiLSTM_V11
Test Accuracy Hybrid: 0.807
